In [ ]:
from pathlib import Path
import textwrap
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xarray as xr
from typing import List, Tuple, Dict, Any


strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    adjust_lightness_array,
    label_from_attrs,
    plot_one_one,
)
from sdm_eurec4a import RepositoryPath

default_colors = set_custom_rcParams()
default_dark_colors = adjust_lightness_array(default_colors, 0.75)

RepoPaths = RepositoryPath("levante")

data_dir = RepoPaths.CLEO_data_dir / Path("output_v4.1")
fig_dir = RepoPaths.fig_dir / Path("paper-v4.1")
fig_dir.mkdir(exist_ok=True, parents=False)

from sdm_eurec4a import data_loading
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

In [ ]:
microphysics = data_loading.MicrophysicsStyles()
microphysics

In [ ]:
selected_microphysics = "condensation"
cleo_dataset = data_loading.CleoDataset(
    data_dir=data_dir,
    microphysics=(selected_microphysics,),
)

cleo_dataset.normalize_gridboxes()

condensation


In [ ]:
ds, ds_sem = cleo_dataset()
ds = ds.where(np.isfinite(ds["relative_humidity"]), drop=True)
ds, ds_sem = ds.sel(microphysics=selected_microphysics), ds_sem.sel(microphysics=selected_microphysics)

In [ ]:
for cloud_id in ds["cloud_id"]:

    m, s = ds["evaporation_rate"].sel(cloud_id=cloud_id), ds_sem["evaporation_rate"].sel(
        cloud_id=cloud_id
    )

    plt.plot(
        m.T,
        ds["normalized_gridbox_coord3"],
        color="grey",
        alpha=0.3,
    )
    # plt.fill_betweenx(
    #     ds["normalized_gridbox_coord3"],
    #     m - s,
    #     m + s,
    #     color = "grey",
    #     alpha = 0.3,
    # );

m2, s2 = mean_and_stderror_of_mean(
    data=ds["evaporation_rate"],
    dims="cloud_id",
    data_std=ds_sem["evaporation_rate"],
)
# plt.fill_betweenx(
#     ds["normalized_gridbox_coord3"].data,
#     m2 - s2,
#     m2 + s2,
#     color="red",
#     alpha=0.5,
# )

m = ds["evaporation_rate"].mean(dim="cloud_id")
s = 1 / np.sqrt(len(ds_sem["cloud_id"])) * (ds_sem["evaporation_rate"] ** 2).sum(dim="cloud_id") ** 0.5
# s = np.sqrt(1 / weights.sum(dim="cloud_id"))
# s = 0
# plt.fill_betweenx(
#     ds["normalized_gridbox_coord3"],
#     m - s,
#     m + s,
#     color="blue",
#     alpha=0.4,
#     zorder=10,
# );
# plt.xscale("log")

### Recompute the evaporation fraction with Florians equation

In [ ]:
rhow = 0.998e3
rhoa = 1.2
eta = 1.85e-5
g = 9.81
nu = eta / rhoa
T = 294.41807507
p = 1e5
Dv0 = 0.211 * (T / 273.15) ** (1.94) * (1013.25e2 / p) * 1e-4  # PK97 (13-3)
Sc = 0.71  # nu/Dv0
gamma = 73e-3
Coo = 0.26
Cgamma = 18.4
lgamma = np.sqrt(gamma / (rhow * g))
kb = 1.380649e-23
Rconst = 8.314
Rv = 461.5
lv = 2.5e6
ka = 26.19e-3


def psat_water(T):
    theta = T - 273.15
    psat = 6.1121e2 * np.exp((18.678 - theta / 234.5) * (theta / (257.14 + theta)))
    return psat


def rhosat_water(T):
    rho = psat_water(T) * 18.01528e-3 / (Rconst * T)
    return rho


Dv = Dv0 / (1 + lv * Dv0 * rhosat_water(T) / (ka * T) * (lv / (Rv * T) - 1))
print(Dv0 / Dv)

3.5957635456780634


In [ ]:
ds, ds_sem = cleo_dataset = data_loading.CleoDataset(
    data_dir=data_dir,
    microphysics=(selected_microphysics,),
)()

ds, ds_sem = ds.sel(microphysics=selected_microphysics), ds_sem.sel(microphysics=selected_microphysics)

ds["radius_bins"].attrs = dict(
    long_name="Radius",
    units="$\\mu m$",
)
radii_label = label_from_attrs(ds["radius_bins"])

condensation


In [ ]:
RH = ds["relative_humidity"].mean() / 100
H = 1000


def theoretical_evaporation_fraction(r0s: xr.DataArray) -> xr.DataArray:
    bU = np.sqrt(8 / 3 * rhow / rhoa * g / 0.5)
    dr52 = 5 / 2 * Dv * H / bU * (1 - RH) * rhosat_water(T) / rhow
    efftheo = 1 - (1 - dr52 / r0s ** (5 / 2)) ** (6 / 5)
    efftheo = np.minimum(efftheo.fillna(1), 1)
    return efftheo


def fv(a, v):
    """Arguments are mass and velocity"""
    Re = 2 * a * np.abs(v) / nu
    x = Sc ** (1 / 3) * Re ** (1 / 2)
    if a < 60e-6:
        return 1 + 0.108 * x**2
    else:
        return 0.78 + 0.308 * x


def fv_xr(a: xr.DataArray, v: xr.DataArray) -> xr.DataArray:
    """Arguments are mass and velocity"""
    Re = 2 * a * np.abs(v) / nu
    x = Sc ** (1 / 3) * Re ** (1 / 2)
    low = 1 + 0.108 * x**2
    high = 0.78 + 0.308 * x

    return xr.where(a < 60e-6, low, high)


def vtlim(a):
    """Terminal velocity in m/s"""
    c1 = Coo ** (1 / 2)
    c2 = (12 * nu / a) ** (1 / 2)
    c3 = (8 * rhow * g * a / (3 * rhoa)) ** (1 / 2)
    return ((np.sqrt(c2**2 + 4 * c1 * c3) - c2) / (2 * c1)) ** 2


def vt(a):
    """Terminal velocity in m/s"""
    c1 = Coo ** (1 / 2) * (1 + Cgamma * (a / lgamma) ** 3) ** (1 / 6)
    c2 = (12 * nu / a) ** (1 / 2)
    c3 = (8 * rhow * g * a / (3 * rhoa)) ** (1 / 2)
    return ((np.sqrt(c2**2 + 4 * c1 * c3) - c2) / (2 * c1)) ** 2


ventilation_coefficient = fv_xr(ds["radius_bins"] * 1e-6, vt(ds["radius_bins"] * 1e-6))

fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(ds["radius_bins"], ventilation_coefficient, label="fv", linestyle="-")
ax.set_xlabel(radii_label)
ax.set_ylabel("fv")
ax.set_ylim(0, 17)
# ax.legend()

(0.0, 17.0)

In [ ]:
evaporation_fraction = theoretical_evaporation_fraction(ds["radius_bins"] * 1e-6)
evaporation_fraction_ventilation = evaporation_fraction * ventilation_coefficient
evaporation_fraction_ventilation: xr.DataArray = np.minimum(
    evaporation_fraction_ventilation.fillna(1), 1
)

plt.plot(
    ds["radius_bins"],
    evaporation_fraction,
    label="no ventilation",
)
plt.plot(
    ds["radius_bins"],
    evaporation_fraction_ventilation,
    label="ventilation",
)
plt.xlabel(radii_label)
plt.xlim(1e1, None)
plt.ylabel("Evaporation fraction")
plt.loglog()
plt.legend()

In [ ]:
m = ds["mass_represented_temporal_mean"].sel(gridbox=ds["max_gridbox"]).drop_vars("gridbox")
weights = m / m.sum("radius_bins")

ef_no_ventilation = (evaporation_fraction * weights).sum(dim="radius_bins") * 100
ef_no_ventilation.attrs = dict(
    long_name="Approx. EF without ventilation",
    units="%",
)
ef_ventilation = (evaporation_fraction_ventilation * weights).sum(dim="radius_bins") * 100
ef_ventilation.attrs = dict(
    long_name="Approx. EF with ventilation",
    units="%",
)

In [ ]:
y = ds["evaporation_fraction"]
y.attrs["long_name"] = "CLEO evaporation fraction"

xlabel = "Approx. " + label_from_attrs(y)
ylabel = label_from_attrs(y)
# xlabel = "\n".join(textwrap.wrap(xlabel, 20))
ywraps = textwrap.wrap(ylabel, 20)
ywraps = [ywraps[0], " ".join(ywraps[1:])]
ylabel = "\n".join(ywraps)

fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
axs: Tuple[plt.Axes, plt.Axes] = axs

axs[0].scatter(
    ef_no_ventilation,
    y,
    marker="+",
    label=label_from_attrs(ef_no_ventilation, return_units=False),
)
axs[0].scatter(
    ef_ventilation,
    y,
    marker="x",
    label=label_from_attrs(ef_ventilation, return_units=False),
)

axs[0].set_xlabel(xlabel)
axs[0].set_ylabel(ylabel)
axs[0].legend(loc="upper left")
axs[0].loglog()
title = "Comparison of evaporation fraction between CLEOs results and theoretical approximation."
axs[0].set_title("\n".join(textwrap.wrap(title, 40)))
plot_one_one(axs[0], color="grey", linestyle="--")

hist_data = [ef_no_ventilation, ef_ventilation, y]
hist_labels = [label_from_attrs(d, return_units=False) for d in hist_data]
hist_colors = default_colors[: len(hist_data)]

bins = np.linspace(0, 100, 20)
# bins = np.geomspace(2e-1, 100, 20)
n, bins, patches = axs[1].hist(
    hist_data,
    bins=bins,
    label=hist_labels,
    color=hist_colors,
    edgecolor=hist_colors,
    alpha=0.3,
    histtype="stepfilled",
)
n, bins, patches = axs[1].hist(
    hist_data,
    bins=bins,
    # label = hist_labels,
    color=hist_colors,
    alpha=0.7,
    linewidth=2,
    linestyle="-",
    histtype="step",
)
axs[1].legend()
axs[1].set_xlabel(label_from_attrs(y))
axs[1].set_ylabel("Counts")
# axs[1].set_xscale("log")
axs[1].set_title("Histogram of evaporation fraction")

fig.tight_layout()

In [ ]:
y = ds["evaporation_fraction"]
y.attrs["long_name"] = "CLEO evaporation fraction"

xlabel = "Approx. " + label_from_attrs(y)
ylabel = label_from_attrs(y)
# xlabel = "\n".join(textwrap.wrap(xlabel, 20))
ywraps = textwrap.wrap(ylabel, 20)
ywraps = [ywraps[0], " ".join(ywraps[1:])]
ylabel = "\n".join(ywraps)

fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
axs: Tuple[plt.Axes, plt.Axes] = axs

axs[0].scatter(
    ef_no_ventilation,
    y,
    marker="+",
    label=label_from_attrs(ef_no_ventilation, return_units=False),
)
axs[0].scatter(
    ef_ventilation,
    y,
    marker="x",
    label=label_from_attrs(ef_ventilation, return_units=False),
)

axs[0].set_xlabel(xlabel)
axs[0].set_ylabel(ylabel)
axs[0].legend(loc="upper left")
# axs[0].loglog()
title = "Comparison of evaporation fraction between CLEOs results and theoretical approximation."
axs[0].set_title("\n".join(textwrap.wrap(title, 40)))
plot_one_one(axs[0], color="grey", linestyle="--")

hist_data = [ef_no_ventilation, ef_ventilation, y]
hist_labels = [label_from_attrs(d, return_units=False) for d in hist_data]
hist_colors = default_colors[: len(hist_data)]

bins = np.linspace(0, 100, 20)
# bins = np.geomspace(2e-1, 100, 20)
n, bins, patches = axs[1].hist(
    hist_data,
    bins=bins,
    label=hist_labels,
    color=hist_colors,
    edgecolor=hist_colors,
    alpha=0.3,
    histtype="stepfilled",
)
n, bins, patches = axs[1].hist(
    hist_data,
    bins=bins,
    # label = hist_labels,
    color=hist_colors,
    alpha=0.7,
    linewidth=2,
    linestyle="-",
    histtype="step",
)
axs[1].legend()
axs[1].set_xlabel(label_from_attrs(y))
axs[1].set_ylabel("Counts")
# axs[1].set_xscale("log")
axs[1].set_title("Histogram of evaporation fraction")

fig.tight_layout()

In [ ]:
y = ds["evaporation_fraction"]
y.attrs["long_name"] = "CLEO evaporation fraction"

xlabel = "Approx. " + label_from_attrs(y)
ylabel = label_from_attrs(y)
# xlabel = "\n".join(textwrap.wrap(xlabel, 20))
ywraps = textwrap.wrap(ylabel, 20)
ywraps = [ywraps[0], " ".join(ywraps[1:])]
ylabel = "\n".join(ywraps)

fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
axs: Tuple[plt.Axes, plt.Axes] = axs

axs[0].scatter(
    ef_no_ventilation,
    y,
    marker="+",
    label=label_from_attrs(ef_no_ventilation, return_units=False),
)
axs[0].scatter(
    ef_ventilation,
    y,
    marker="x",
    label=label_from_attrs(ef_ventilation, return_units=False),
)

axs[0].set_xlabel(xlabel)
axs[0].set_ylabel(ylabel)
axs[0].legend(loc="upper left")
# axs[0].loglog()
title = "Comparison of evaporation fraction between CLEOs results and theoretical approximation."
axs[0].set_title("\n".join(textwrap.wrap(title, 40)))
plot_one_one(axs[0], color="grey", linestyle="--")

hist_data = [ef_no_ventilation, ef_ventilation, y]
hist_labels = [label_from_attrs(d, return_units=False) for d in hist_data]
hist_colors = default_colors[: len(hist_data)]

bins = np.arange(0, 101, 10)
# bins = np.geomspace(2e-1, 100, 20)
n, bins, patches = axs[1].hist(
    hist_data,
    bins=bins,
    label=hist_labels,
    color=hist_colors,
    edgecolor=hist_colors,
    alpha=0.3,
    histtype="stepfilled",
)
n, bins, patches = axs[1].hist(
    hist_data,
    bins=bins,
    # label = hist_labels,
    color=hist_colors,
    alpha=0.7,
    linewidth=2,
    linestyle="-",
    histtype="step",
)
axs[1].legend()
axs[1].set_xlabel(label_from_attrs(y))
axs[1].set_ylabel("Counts")
# axs[1].set_xscale("log")
axs[1].set_title("Histogram of evaporation fraction")

fig.tight_layout()

In [ ]:
for data in hist_data:
    plt.scatter(
        ds["cloud_mass_radius_mean"],
        data,
        marker=".",
        label=label_from_attrs(data, return_units=False),
    )

plt.plot(
    ds["radius_bins"],
    1e2 * evaporation_fraction,
    label="no ventilation",
)
plt.plot(
    ds["radius_bins"],
    1e2 * evaporation_fraction_ventilation,
    label="ventilation",
)
plt.xlabel(radii_label)

plt.loglog()
plt.legend(loc="lower left")